# Skin Lesion Classifier
This model is trained on close up images of the following diseases:

- "Bowen's disease"                 | very early form of skin cancer 
- "basal cell carcinoma"            | basal-cell cancer or white skin cancer 
- "benign keratosis-like lesions"   | non-cancerous 
- "dermatofibroma"                  | non-cancerous 
- "melanoma",                       | black skin cancer 
- "melanocytic nevi",               | mole non-cancerous 
- "vascular lesions",               | skin condition 

You can upload a picture of any mole or other skin condition and see what the model predicts.

For most accurate results:
    The picture should be **as close as possible to the skin and still be sharp** aswell as capture the unusaul part of the skin **right in the middle**.

**IMPORTANT**: The Prediction of this model **must be by no means be true**. If you are unsure about some parts of your skin you should **see a doctor** and **not** rely on this model.

In [122]:
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila

from fastai.vision.all import load_learner, PILImage, CropPad
from fastai.vision.widgets import Path, widgets, platform, VBox
from IPython.display import display

import pathlib
plt = platform.system()
if plt == 'Linux': pathlib.WindowsPath = pathlib.PosixPath
elif plt == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

In [123]:
path = Path('./models/skin_disease_50_res18_9217.pkl')
def get_label_from_dict():
    pass
learn_inf = load_learner(path, cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred_1 = widgets.Label()
lbl_pred_2 = widgets.Label()
checkbox_crop = widgets.Checkbox()
checkbox_crop.indent = False
checkbox_crop.description  = "Center-Crop your Image?"
checkbox_crop.value  = True

In [124]:
def crop_middle_part(img):
    width, height = img.size
    mid_width = width//2
    mid_height = height//2
    offset_from_mid = 225
    
    if min(mid_width, mid_height) <  offset_from_mid:
        print("Image to small to crop.")
        return img
        
    left = mid_width - offset_from_mid
    top = mid_height - offset_from_mid
    right = mid_width + offset_from_mid
    bottom = mid_height + offset_from_mid
    
    return img.crop((left, top, right, bottom))

def invert_softmax_prob(probs):
    probs = probs.log()
    probs = probs - probs.min() + 1
    return probs.div(probs.sum()) 

def sorted_classes_logged_probs(probs, learn):
    real_probs = invert_softmax_prob(probs)
    classes_probs = [(x,prob.item()) for x, prob in zip(learn.dls.vocab.items, real_probs)]
    classes_probs.sort(key= lambda x : x[1], reverse=True)
    return classes_probs

def on_click_btn_upload(change):
    lbl_pred_1.value = "Processing"
    lbl_pred_2.value = ""
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    if checkbox_crop.value == True:
        img = crop_middle_part(img)
        with out_pl: 
            display(img.to_thumb(128,128))
        img = img.to_bytes_format()
    else:
        with out_pl: 
            display(img.to_thumb(128,128))

    pred,pred_idx,probs = learn_inf.predict(img)
    
    sorted_classes = sorted_classes_logged_probs(probs, learn_inf)

    lbl_pred_1.value = f'First Guess: {sorted_classes[0][0]} ! I am {sorted_classes[0][1]*100:.02f}% sure.'
    lbl_pred_2.value = f'Second Guess: {sorted_classes[1][0]} ! I am {sorted_classes[1][1]*100:.02f}% sure.'
    

In [125]:
btn_upload.observe(on_click_btn_upload, names=['data'])

In [126]:
display(VBox([widgets.Label('Upload Skin Picture!'), btn_upload, checkbox_crop,  out_pl, lbl_pred_1, lbl_pred_2]))